# Analysis of Flight Delays
## by Robert Hofstetter

## Preliminary Wrangling

> The data for this analysis was obtained from the [Bureau of Transportation Staticis](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236). All domestic United States regions for years 2018 and 2019 were download by month and concatenated together outside of this notebook into two separate files for each year. It is a rather large set of data so I've separated the wrangling in order to speed processing of cells in the analysis. The data is then aggregated by day and saved to file.

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os.path
from scipy import stats

%matplotlib inline

# Gather
Check if the cleaned file exists and load it directly. Otherwise, load one file for each year of flight data

In [2]:
flights_2018 = pd.read_csv('flights_2018.csv')
flights_2019 = pd.read_csv('flights_2019.csv')
delayed_flights = pd.concat([flights_2018, flights_2019])

# Assess
Examine the cleanliness and tidiness of the data

In [3]:
delayed_flights.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,...,ARR_DELAY,CANCELLED,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 29
0,2018,4,11,18,7,2018-11-18,9E,N909XJ,5044,DTW,...,-21.0,0.0,82.0,534.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,4,11,20,2,2018-11-20,9E,N600LR,5044,DTW,...,-4.0,0.0,83.0,534.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,4,11,21,3,2018-11-21,9E,N903XJ,5044,DTW,...,-26.0,0.0,85.0,534.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,4,11,22,4,2018-11-22,9E,N326PQ,5044,DTW,...,5.0,0.0,81.0,534.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,4,11,23,5,2018-11-23,9E,N324PQ,5044,DTW,...,-5.0,0.0,92.0,534.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
delayed_flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14635483 entries, 0 to 7422036
Data columns (total 30 columns):
YEAR                   int64
QUARTER                int64
MONTH                  int64
DAY_OF_MONTH           int64
DAY_OF_WEEK            int64
FL_DATE                object
OP_UNIQUE_CARRIER      object
TAIL_NUM               object
OP_CARRIER_FL_NUM      int64
ORIGIN                 object
ORIGIN_CITY_NAME       object
ORIGIN_STATE_ABR       object
DEST                   object
DEST_CITY_NAME         object
DEST_STATE_ABR         object
CRS_DEP_TIME           int64
DEP_TIME               float64
DEP_DELAY              float64
CRS_ARR_TIME           int64
ARR_TIME               float64
ARR_DELAY              float64
CANCELLED              float64
AIR_TIME               float64
DISTANCE               float64
CARRIER_DELAY          float64
WEATHER_DELAY          float64
NAS_DELAY              float64
SECURITY_DELAY         float64
LATE_AIRCRAFT_DELAY    float64
Unnamed: 29 

In [5]:
delayed_flights[['CRS_DEP_TIME','DEP_TIME']].head()

,CRS_DEP_TIME,DEP_TIME
0,1555,1550.0
1,1555,1605.0
2,1555,1549.0
3,1555,1550.0
4,1555,1551.0


In [6]:
delayed_flights['Unnamed: 29'].unique()

array([nan])

In [7]:
delayed_flights['DEP_DELAY'].unique()

array([  -5.,   10.,   -6., ..., 1637., 1566., 1387.])

In [8]:
delayed_flights['ARR_DELAY'].unique()

array([ -21.,   -4.,  -26., ..., 1448., 1354., 1398.])

In [9]:
delayed_flights['OP_UNIQUE_CARRIER'].value_counts()

WN    2716498
DL    1941269
AA    1863594
OO    1610582
UA    1247475
YX     645239
MQ     623008
B6     602421
OH     567761
AS     510577
9E     503049
YV     443026
NK     381023
EV     337573
F9     255578
G4     201526
HA     167614
VX      17670
Name: OP_UNIQUE_CARRIER, dtype: int64

In [10]:
delayed_flights[['DEP_DELAY','ARR_DELAY','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY']].describe()

,DEP_DELAY,ARR_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
count,1.438814e+07,1.434464e+07,2.741963e+06,2.741963e+06,2.741963e+06,2.741963e+06,2.741963e+06
mean,1.045305e+01,5.234164e+00,2.030295e+01,3.720537e+00,1.624139e+01,9.482805e-02,2.653755e+01
std,4.697056e+01,4.906834e+01,6.266330e+01,3.121780e+01,3.785268e+01,3.268848e+00,5.168649e+01
min,-1.220000e+02,-1.200000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,-5.000000e+00,-1.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,-2.000000e+00,-6.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,3.000000e+00
75%,7.000000e+00,8.000000e+00,1.700000e+01,0.000000e+00,2.000000e+01,0.000000e+00,3.200000e+01
max,2.710000e+03,2.695000e+03,2.695000e+03,2.692000e+03,1.848000e+03,1.078000e+03,2.454000e+03


In [11]:
delayed_flights[delayed_flights['DEP_DELAY'] == 2710]

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,...,ARR_DELAY,CANCELLED,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 29
4329216,2018,3,7,5,4,2018-07-05,OO,N264SY,4690,CLT,...,2692.0,0.0,79.0,541.0,0.0,2692.0,0.0,0.0,0.0,NaN
822649,2019,4,12,12,4,2019-12-12,OO,N727SK,3253,SYR,...,2695.0,0.0,107.0,607.0,2695.0,0.0,0.0,0.0,0.0,NaN


# Data Assessment
The data is fairly clean and tidy but there are a few things that need to be addressed:
1. There are cancelled and early flights which do not register a delay
2. Double counting of delays from including records having late aircraft
3. There is an unnamed column that has no data and thus no value
4. Arrival time and scheduled arrival time are not the same data types
5. Departure time and scheduled departure time are not the same data types
6. Columns TAIL_NUM and OP_CARRIER_FL_NUM are not useful for this exercise
7. Delays greater than a day are really cancelled flights
8. There are early departures and arrivals represented as negative values

# Clean

In [12]:
# Fix the data types of the scheduled times to match the data type of actual times
delayed_flights['CRS_ARR_TIME'] = delayed_flights['ARR_TIME'].astype('float')
delayed_flights['CRS_DEP_TIME'] = delayed_flights['DEP_TIME'].astype('float')

In [13]:
# Drop all delays greater than 1 day because those would really be considered cancelled and rebooked
delayed_flights = delayed_flights[delayed_flights['DEP_DELAY'] < 1440]

In [14]:
# Drop cancelled flights
delayed_flights = delayed_flights[delayed_flights['CANCELLED'] == 0]

In [15]:
# Retain only flights that had a delay in either depature or arrival
delayed_flights = delayed_flights[(delayed_flights['DEP_DELAY'] > 0) | (delayed_flights['ARR_DELAY'] > 0)]

In [16]:
# Drop the late aircraft delays that generally indicate the previous flight was delayed to avoid double counting
delayed_flights = delayed_flights[delayed_flights['LATE_AIRCRAFT_DELAY'] == 0]

In [17]:
# Drop unused columns
delayed_flights = delayed_flights.drop(columns=['Unnamed: 29', 'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'CANCELLED', 'LATE_AIRCRAFT_DELAY'])

In [18]:
delayed_flights[['DEP_DELAY','ARR_DELAY','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY']].describe()

,DEP_DELAY,ARR_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY
count,1.321839e+06,1.321839e+06,1.321839e+06,1.321839e+06,1.321839e+06,1.321839e+06
mean,4.906143e+01,5.999704e+01,3.007885e+01,5.215891e+00,2.456486e+01,1.374396e-01
std,9.233441e+01,8.811010e+01,7.619636e+01,3.804508e+01,4.588410e+01,3.861963e+00
min,-4.700000e+01,1.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,2.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.500000e+01,3.300000e+01,1.000000e+00,0.000000e+00,1.600000e+01,0.000000e+00
75%,5.900000e+01,6.300000e+01,3.000000e+01,0.000000e+00,2.900000e+01,0.000000e+00
max,1.439000e+03,1.540000e+03,1.458000e+03,1.431000e+03,1.515000e+03,1.048000e+03


In [19]:
print(delayed_flights['DEP_DELAY'].min())
print(delayed_flights['DEP_DELAY'].max())
print(delayed_flights['ARR_DELAY'].min())
print(delayed_flights['ARR_DELAY'].max())

-47.0
1439.0
15.0
1540.0


In [20]:
# For analysis of delays we cannot have negative values which really represent the opposite of a delay
delayed_flights.loc[delayed_flights['DEP_DELAY'] < 0, 'DEP_DELAY'] = 0
delayed_flights.loc[delayed_flights['ARR_DELAY'] < 0, 'ARR_DELAY'] = 0
print(delayed_flights['DEP_DELAY'].min())
print(delayed_flights['DEP_DELAY'].max())
print(delayed_flights['ARR_DELAY'].min())
print(delayed_flights['ARR_DELAY'].max())

0.0
1439.0
15.0
1540.0


In [21]:
# Create an aggregation of count of flights by carrier and mean departure delay
carrier_agg = delayed_flights.groupby(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST']) \
    .agg({'DEP_DELAY':['count', 'mean', 'min', 'max'], 'ARR_DELAY':['mean', 'min', 'max'], 'DISTANCE':['mean']})

In [22]:
# Rename the columns and reset the index after aggregating
carrier_agg.columns = ['FLIGHTS', 'DEP_DELAY_MEAN', 'DEP_DELAY_MIN', 'DEP_DELAY_MAX', 'ARR_DELAY_MEAN', 'ARR_DELAY_MIN', 'ARR_DELAY_MAX', 'MEAN_DISTANCE']
carrier_agg.reset_index()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,ORIGIN,DEST,FLIGHTS,DEP_DELAY_MEAN,DEP_DELAY_MIN,DEP_DELAY_MAX,ARR_DELAY_MEAN,ARR_DELAY_MIN,ARR_DELAY_MAX,MEAN_DISTANCE
0,2018,1,1,1,9E,AGS,ATL,1,32.0,32.0,32.0,30.0,30.0,30.0,143.0
1,2018,1,1,1,9E,ATL,AEX,1,55.0,55.0,55.0,43.0,43.0,43.0,500.0
2,2018,1,1,1,9E,ATL,BMI,1,0.0,0.0,0.0,53.0,53.0,53.0,533.0
3,2018,1,1,1,9E,ATL,CHS,1,80.0,80.0,80.0,72.0,72.0,72.0,259.0
4,2018,1,1,1,9E,ATL,CSG,1,102.0,102.0,102.0,139.0,139.0,139.0,83.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060601,2019,12,31,2,YX,PWM,DCA,1,191.0,191.0,191.0,219.0,219.0,219.0,482.0
1060602,2019,12,31,2,YX,PWM,PHL,1,123.0,123.0,123.0,181.0,181.0,181.0,364.0
1060603,2019,12,31,2,YX,RDU,EWR,1,0.0,0.0,0.0,16.0,16.0,16.0,416.0
1060604,2019,12,31,2,YX,RDU,RSW,1,0.0,0.0,0.0,26.0,26.0,26.0,667.0


In [23]:
carrier_agg[carrier_agg['FLIGHTS'] > 10]

FLIGHTS  \
YEAR MONTH DAY_OF_MONTH DAY_OF_WEEK OP_UNIQUE_CARRIER ORIGIN DEST            
2018 1     23           2           DL                ATL    LGA        11   
     2     21           3           AS                SEA    ANC        12   
     3     27           2           AS                SEA    ANC        11   
     4     3            2           DL                MSP    ATL        11   
           25           3           DL                ATL    LGA        13   
                                    UA                ORD    LGA        12   
           30           1           AS                SEA    ANC        12   
     5     24           4           AS                SEA    LAX        11   
     8     21           2           AA                ORD    LGA        11   
           25           6           AS                SEA    ANC        13   
     9     10           1           DL                ATL    LGA        12   
     10    18           4           YX                DCA    LGA        11   
           22           1           AS                LAX    SEA        12   
     11    6            2           YX                DCA    LGA        11   
           9            5           DL                MCO    ATL        11   
           13           2           UA                ORD    LGA        12   
           14           3           DL                LGA    ATL        13   
           28           3           DL                ATL    LGA        12   
                                    UA                ORD    LGA        13   
     12    13           4           YX                LGA    DCA        11   
           18           2           UA                ORD    LGA        12   
                                    YX                DCA    LGA        13   
           21           5           AA                DFW    LGA        11   
2019 1     10           4           DL                ATL    LGA        14   
           17           4           YX                DCA    LGA        11   
           22           2           YX                DCA    LGA        13   
           23           3           YX                DCA    LGA        11   
     2     5            2           AS                SEA    ANC        11   
     3     1            5           DL                ATL    LGA        11   
                                    UA                ORD    LGA        11   
     4     22           1           YX                DCA    LGA        11   
     5     29           3           DL                ATL    LAX        12   
     6     16           7           AA                ORD    DFW        11   
     9     12           4           AA                DFW    ORD        11   
     11    18           1           DL                ATL    LGA        11   
           27           3           B6                JFK    LAX        12   
           30           6           AA                ORD    DFW        11   
     12    2            1           AS                SEA    SFO        11   
           15           7           B6                JFK    LAX        13   

                                                                   DEP_DELAY_MEAN  \
YEAR MONTH DAY_OF_MONTH DAY_OF_WEEK OP_UNIQUE_CARRIER ORIGIN DEST                   
2018 1     23           2           DL                ATL    LGA        51.000000   
     2     21           3           AS                SEA    ANC         9.000000   
     3     27           2           AS                SEA    ANC         4.363636   
     4     3            2           DL                MSP    ATL         9.727273   
           25           3           DL                ATL    LGA        50.692308   
                                    UA                ORD    LGA        79.166667   
           30           1           AS                SEA    ANC         2.000000   
     5     24           4           AS                SEA    LAX        30.272727   
     8     21     

In [24]:
carrier_agg.to_csv('delayed_flights.csv', index=True)